In [1]:
import numpy as np
from scipy import signal
from sympy import *
import eom2js

def disp(name: str, symbol: Symbol):
    display(Eq(Dummy(name), symbol, evaluate=False))

def mateq2list(lhs: Matrix, rhs: Matrix):
    L = []
    for i in range(lhs.rows):
        L.append(Eq(lhs[i], rhs[i], evaluate=False))
    return L

In [2]:
m = symbols("m")
T1, T2 = symbols("T1 T2")
a = symbols("alpha")

w, h = symbols("w, h")

# Fuerzas y Torque body frame
F_bf = Matrix([
    (T1-T2)*sin(a),
    (-T1-T2)*cos(a)
])
T = (T1-T2)*(cos(a)*w/2 - sin(a)*h/2)

disp("F", F_bf)
disp("T", T)

Eq(_F, Matrix([
[ (T1 - T2)*sin(alpha)],
[(-T1 - T2)*cos(alpha)]]))

Eq(_T, (T1 - T2)*(-h*sin(alpha)/2 + w*cos(alpha)/2))

In [3]:
theta = symbols("theta", cls=Function)
t = symbols("t")
g = symbols("g")
# Fuerzas en sistema inercial
R = Matrix([
    [cos(theta(t)), -sin(theta(t))],
    [sin(theta(t)), cos(theta(t))]
])

F = R*F_bf + Matrix([0, m*g])

disp("F", F)

Eq(_F, Matrix([
[     -(-T1 - T2)*sin(theta(t))*cos(alpha) + (T1 - T2)*sin(alpha)*cos(theta(t))],
[g*m + (-T1 - T2)*cos(alpha)*cos(theta(t)) + (T1 - T2)*sin(alpha)*sin(theta(t))]]))

In [4]:
x, y = symbols("x, y", cls=Function)

I = 1/12*m*(h**2+w**2)

eom = eom2js.EOM(t, x(t), y(t), theta(t))
eom.param = [g, m, a, w, h]
eom.input = [T1, T2]

# Ecuaciones del movimiento
eqs = mateq2list(diff(Matrix([x(t), y(t)]), t, 2), F/m)
eqs += [Eq(diff(theta(t), t, 2), T/I)]

eom.add_eq(*eqs)
eom.printjs("js/eom.js")

for eq in eom.eqs: display(eq)

Derivative(x(t), t)

Derivative(y(t), t)

Derivative(theta(t), t)

T1*sin(alpha)*cos(theta(t))/m + T1*sin(theta(t))*cos(alpha)/m - T2*sin(alpha)*cos(theta(t))/m + T2*sin(theta(t))*cos(alpha)/m

T1*sin(alpha)*sin(theta(t))/m - T1*cos(alpha)*cos(theta(t))/m - T2*sin(alpha)*sin(theta(t))/m - T2*cos(alpha)*cos(theta(t))/m + g

-6.0*T1*h*sin(alpha)/(h**2*m + m*w**2) + 6.0*T1*w*cos(alpha)/(h**2*m + m*w**2) + 6.0*T2*h*sin(alpha)/(h**2*m + m*w**2) - 6.0*T2*w*cos(alpha)/(h**2*m + m*w**2)

In [5]:
# Linealizar y espacio estado
linear = []
for eq in eqs:
	eq = eq.subs(sin(theta(t)), 0)
	eq = eq.subs(cos(theta(t)), 1)
	eq = eq.subs(g, 0)
	linear += [eq]
linear[0] = Eq(diff(x(t), t, 2), theta(t)*g)

eom_linear = eom2js.EOM(t, x(t), y(t), theta(t))
eom_linear.param = [g, m, a, w, h]
eom_linear.input = [T1, T2]

eom_linear.add_eq(*linear)
for eq in eom_linear.eqs: display(eq)
eom_linear.printjs("js/eom_linear.js")

A = linear_eq_to_matrix(eom_linear.eqs, eom.state())[0]
B = linear_eq_to_matrix(eom_linear.eqs, [T1, T2])[0]
display(A, B)

Derivative(x(t), t)

Derivative(y(t), t)

Derivative(theta(t), t)

g*theta(t)

-T1*cos(alpha)/m - T2*cos(alpha)/m

-6.0*T1*h*sin(alpha)/(h**2*m + m*w**2) + 6.0*T1*w*cos(alpha)/(h**2*m + m*w**2) + 6.0*T2*h*sin(alpha)/(h**2*m + m*w**2) - 6.0*T2*w*cos(alpha)/(h**2*m + m*w**2)

Matrix([
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 1],
[0, 0, g, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0]])

Matrix([
[                                                                       0,                                                                       0],
[                                                                       0,                                                                       0],
[                                                                       0,                                                                       0],
[                                                                       0,                                                                       0],
[                                                           -cos(alpha)/m,                                                           -cos(alpha)/m],
[-6.0*h*sin(alpha)/(h**2*m + m*w**2) + 6.0*w*cos(alpha)/(h**2*m + m*w**2), 6.0*h*sin(alpha)/(h**2*m + m*w**2) - 6.0*w*cos(alpha)/(h**2*m + m*w**2)]])

In [67]:
vals = (g, a, m, w, h)
A_num = lambdify(vals, A, modules="numpy")(9.81, np.pi/6, 24, 0.2, 0.3)
B_num = lambdify(vals, B, modules="numpy")(9.81, np.pi/6, 24, 0.2, 0.3)

poles = np.array([-0.1, -0.2, -0.3, -0.5, -0.6, -0.7])
K = signal.place_poles(A_num, B_num, poles)

print(eom.printer.doprint((-Matrix(K.gain_matrix)*Matrix(eom.state()))[0]))
print(eom.printer.doprint((-Matrix(K.gain_matrix)*Matrix(eom.state()))[1]))

-8.99508687596126*s.theta - 0.0101699793138771*s.x + 1.66276590023259*s.y - 17.3871490145351*s.dtheta - 0.179184595947719*s.dx + 11.0851165329715*s.dy
10.2765553326191*s.theta + 0.0138150694274782*s.x + 1.66277209557984*s.y + 18.4670683818401*s.dtheta + 0.222850477529636*s.dx + 11.0851332514327*s.dy
